In [1]:
import pandas as pd
import warnings
import locale
import unicodedata
import numpy as np
from datetime import datetime

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [2]:
%%sql
SELECT 
    data, 
    numeroNotaFiscal, 
    destino, 
    procedencia, 
    cad_cidades.estado as estado,
    produto, 
    unidade, 
    unidade_peso, 
    quantidade, 
    volume,
    cad_usuarios.tipo,
    nome_usuario_sistema
FROM 
    divtec_notasfiscais
LEFT JOIN
    cad_usuarios
        ON cad_usuarios.matricula_nome = divtec_notasfiscais.destino
LEFT JOIN
    cad_cidades ON divtec_notasfiscais.procedencia = cad_cidades.cidade
WHERE data >='2024-07-01'

,data,numeroNotaFiscal,destino,procedencia,estado,produto,unidade,unidade_peso,quantidade,volume,tipo,nome_usuario_sistema
0,2024-07-22,66122,5500971-CHS COMERCIO DE FRUTAS E,0152098-GOIANIA (GO),GOIAS,07404-TOMATE CAQUI LONGA VIDA,KG,1.000,13860.000,13860.000,ATACADISTA,Andressa
1,2024-07-22,317104,5500893-TROPICAL COMERCIO DE PRO,0143420-TRES FORQUILHAS (RS),RIO GRANDE DO SUL,08102-CENOURA,CX,20.000,210.000,4200.000,ATACADISTA,Mateus
2,2024-07-22,317104,5500893-TROPICAL COMERCIO DE PRO,0143420-TRES FORQUILHAS (RS),RIO GRANDE DO SUL,07802-BETERRABA,CX,20.000,70.000,1400.000,ATACADISTA,Mateus
3,2024-07-22,210965,5500893-TROPICAL COMERCIO DE PRO,0143370-MARIANA PIMENTEL (RS),RIO GRANDE DO SUL,10101-BATATA-DOCE,CX,20.000,150.000,3000.000,ATACADISTA,Mateus
4,2024-07-22,931754,5501039-GEAZI NEVES DA SILVA,0143370-MARIANA PIMENTEL (RS),RIO GRANDE DO SUL,10101-BATATA-DOCE,KG,1.000,8000.000,8000.000,ATACADISTA,Mateus
...,...,...,...,...,...,...,...,...,...,...,...,...
50692,2024-09-12,712683,DANIEL LUCIETTO SCHMITZ,0143314-SAO JOSE DO HORTENCIO (RS),RIO GRANDE DO SUL,01406-LARANJA SUCO,CX,20.000,50.000,1000.000,PRODUTOR,Pedro
50693,2024-09-12,712683,DANIEL LUCIETTO SCHMITZ,0143314-SAO JOSE DO HORTENCIO (RS),RIO GRANDE DO SUL,03201-TANGERINA COMUM / CAI,CX,20.000,15.000,300.000,PRODUTOR,Pedro
50694,2024-09-12,712683,DANIEL LUCIETTO SCHMITZ,0143314-SAO JOSE DO HORTENCIO (RS),RIO GRANDE DO SUL,08501-MANDIOCA,CX,20.000,10.000,200.000,PRODUTOR,Pedro
50695,2024-09-12,712683,DANIEL LUCIETTO SCHMITZ,0143314-SAO JOSE DO HORTENCIO (RS),RIO GRANDE DO SUL,04501-ALFACE,DZ,4.200,60.000,252.000,PRODUTOR,Pedro


## Cotação

In [3]:
%%sql
SELECT
    data,
    produto,
    mais_frequente
FROM 
    divtec_cotacao

,data,produto,mais_frequente
0,2024-01-04,00101-ABACATE,5.000
1,2024-01-09,00101-ABACATE,6.000
2,2024-01-11,00101-ABACATE,6.000
3,2024-01-16,00101-ABACATE,6.000
4,2024-01-18,00101-ABACATE,4.440
...,...,...,...
45623,2023-12-28,54801-VERDES (SAMAMBAIA PRETA),15.000
45624,2023-12-28,51801-VIOLETA,5.900
45625,2023-12-28,51806-VIOLETA MINI 15UN,45.000
45626,2023-01-05,06901-MORANGA CABOTIA PRETA,3.000


In [4]:
cotacao['data'] = cotacao['data'].astype('datetime64[ns]')

## Volume

In [21]:
dados = df.copy(deep=True)
dados['data'] = dados['data'].astype('datetime64[ns]')
dados['dia da semana'] = dados['data'].dt.day_name()

In [22]:
# dados = dados.loc[dados['dia da semana'] == 'Monday']
dados = dados.loc[dados['tipo'] == 'PRODUTOR']

In [23]:
dados = dados[['data','destino','produto','volume','dia da semana']]
dados = dados.merge(cotacao, on=['data','produto'], how='left')
dados['valor comercializado'] = round((dados['mais_frequente'] * dados['volume']),2)
dados

,data,destino,produto,volume,dia da semana,mais_frequente,valor comercializado
0,2024-07-22,0101132-LUIZ ARTUR EBERHARDT JAC,06003-ABOBRINHA ITALIANA,45.000,Monday,4.000,180.000
1,2024-07-22,0101132-LUIZ ARTUR EBERHARDT JAC,07802-BETERRABA,600.000,Monday,3.890,2334.000
2,2024-07-22,0101132-LUIZ ARTUR EBERHARDT JAC,04702-BROCOLIS,240.000,Monday,45.000,10800.000
3,2024-07-22,0101132-LUIZ ARTUR EBERHARDT JAC,08102-CENOURA,1000.000,Monday,3.890,3890.000
4,2024-07-22,0101132-LUIZ ARTUR EBERHARDT JAC,10601-COUVE-FLOR,480.000,Monday,50.000,24000.000
...,...,...,...,...,...,...,...
29048,2024-09-12,DANIEL LUCIETTO SCHMITZ,01406-LARANJA SUCO,1000.000,Thursday,NaN,NaN
29049,2024-09-12,DANIEL LUCIETTO SCHMITZ,03201-TANGERINA COMUM / CAI,300.000,Thursday,NaN,NaN
29050,2024-09-12,DANIEL LUCIETTO SCHMITZ,08501-MANDIOCA,200.000,Thursday,NaN,NaN
29051,2024-09-12,DANIEL LUCIETTO SCHMITZ,04501-ALFACE,252.000,Thursday,NaN,NaN


Filtrando so produtores

In [24]:
lista_produtores = ['0111609-CRISTIANO LONGO TONIOLLO','0101933-PAULO FERNANDO TOMASINI','0108886-JORGE GERMANO SPINDLER','0107068-JORGE LUIS FEIJO NUNES','0109291-OLAVO BOETTCHER','0113990-JOSE AVELINO MENDES DA S','0116295-BELMAR DO NASCIMENTO FRA','0113169-CARLOS RIVA','0104875-FLAMERIS JOSE STUANI']

In [25]:
dados = dados.loc[dados['destino'].isin(lista_produtores)]

In [26]:
sem_cotacao = dados.loc[dados.mais_frequente.isnull()]
for item in sem_cotacao['produto'].unique():
	print(item)

01702-MACA GOLDEN CX PLAST
04501-ALFACE
04801-CHEIRO VERDE / TEMPERO
05001-COUVE
08501-MANDIOCA
05701-RUCULA
05601-REPOLHO VERDE
05301-ESPINAFRE
08102-CENOURA
07802-BETERRABA
02301-MORANGO
07405-TOMATE CEREJA (MINI TOMATES)
07002-PEPINO SALADA
07201-PIMENTAO
07501-VAGEM
07203-PIMENTAO VERMELHO
01703-MACA GALA CX PLAST
01701-MACA FUJI CX PLAST
03204-TANGERINA MONTENEGRINA
10101-BATATA-DOCE
07003-PEPINO JAPONES
01406-LARANJA SUCO
04702-BROCOLIS
08801-RABANETE MOLHO


In [27]:
destino_produto = dados.groupby(['data','destino','produto','dia da semana'])[['volume','valor comercializado']].sum()
destino_produto = destino_produto.reset_index()
destino_produto.to_excel('Volume e Valor por destino e data.xlsx')

In [9]:
destino = dados.groupby(['destino'])[['volume','valor comercializado']].sum()
destino.to_excel('Volume e Valor por destino - Segunda feiras.xlsx')